<a href="https://colab.research.google.com/github/Deniz-shelby/Studienarbeit/blob/main/Data_manipulation_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
# Load the Drive helper and mount
from google.colab import drive, files

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd
import numpy as np
import time
import random
import pytz
from pytz import timezone
from tzlocal import get_localzone
from datetime import datetime as dt
from datetime import timedelta
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import ProbPlot
import patsy
from scipy import stats
from sklearn import datasets
from IPython.display import display, HTML

#load Json

In [ ]:
with open("/content/drive/My Drive/Studienarbeit/acndata_Caltech_04-06:2019.json", "r", encoding='utf-8') as json_file:
    data_load = json.load(json_file)

#Data Frame erzeugen  



In [ ]:
userID = [ sub['userID'] for sub in data_load ]

connectionTime_preparse = [sub ['connectionTime'] for sub in data_load ]

disconnectTime_preparse = [sub ['disconnectTime'] for sub in data_load ]

doneChargingTime_preparse = [ sub['doneChargingTime'] for sub in data_load ]

kWhDelivered = [sub ['kWhDelivered'] for sub in data_load ]

df_na = pd.DataFrame(list(zip(userID,connectionTime_preparse,disconnectTime_preparse,doneChargingTime_preparse,kWhDelivered)),
                      columns = ['userID','connectionTime_preparse','disconnectTime_preparse','doneChargingTime_preparse','kWhDelivered'])

drop none

In [ ]:
df_pre_parse = df_na.mask(df_na.astype(object).eq('None')).dropna() 
print(df_pre_parse)

         userID  ... kWhDelivered
0     000001083  ...        6.219
1     000000489  ...        0.562
2     000000243  ...        3.870
3     000001093  ...        0.789
4     000000365  ...        4.501
...         ...  ...          ...
2006  000001152  ...       11.865
2007  000000675  ...        8.082
2009  000001133  ...        6.062
2011  000000365  ...        2.502
2014  000000891  ...        1.790

[1735 rows x 5 columns]


# String parse to time & timezone function

---



In [ ]:
# functionen für string-parse & timezone

def string_parse_datetime_list(str_list, timeformat):
  return [dt.strptime(x, timeformat) for x in str_list]

def change_timezone_as_list(datetime_list,new_timezone):
  new_datetime_list = []
  for i in range(len(datetime_list)):
    new_datetime_list.append(datetime_list[i].astimezone(timezone(new_timezone)))
  return new_datetime_list

# Str parse to dt

In [ ]:
timeformat= '%a, %d %b %Y %H:%M:%S %Z'

connectionTime_utc = string_parse_datetime_list(df_pre_parse['connectionTime_preparse'], timeformat)

disconnectTime_utc = string_parse_datetime_list(df_pre_parse['disconnectTime_preparse'], timeformat)

doneChargingTime_utc = string_parse_datetime_list(df_pre_parse['doneChargingTime_preparse'], timeformat)


#create list with date, time intervall and parse

In [ ]:
timeintervall = 15

def daterange(start_date, end_date):
    delta = timedelta(minutes=timeintervall)
    while start_date < end_date:
        yield start_date
        start_date += delta

zeitreihe_utc = []
start_date_zeitreihe = dt(2019, 4, 1, 7, 00, 00)
end_date_zeitreihe = dt(2019, 6, 1, 7, 00, 15)
for single_date in daterange(start_date_zeitreihe, end_date_zeitreihe):
    zeitreihe_utc.append(single_date.strftime("%Y-%m-%d %H:%M:%S"))
zeitreihe_utc = [dt.strptime(x,'%Y-%m-%d %H:%M:%S') for x in zeitreihe_utc]
len(zeitreihe_utc)
print(zeitreihe_utc)

[datetime.datetime(2019, 4, 1, 7, 0), datetime.datetime(2019, 4, 1, 7, 15), datetime.datetime(2019, 4, 1, 7, 30), datetime.datetime(2019, 4, 1, 7, 45), datetime.datetime(2019, 4, 1, 8, 0), datetime.datetime(2019, 4, 1, 8, 15), datetime.datetime(2019, 4, 1, 8, 30), datetime.datetime(2019, 4, 1, 8, 45), datetime.datetime(2019, 4, 1, 9, 0), datetime.datetime(2019, 4, 1, 9, 15), datetime.datetime(2019, 4, 1, 9, 30), datetime.datetime(2019, 4, 1, 9, 45), datetime.datetime(2019, 4, 1, 10, 0), datetime.datetime(2019, 4, 1, 10, 15), datetime.datetime(2019, 4, 1, 10, 30), datetime.datetime(2019, 4, 1, 10, 45), datetime.datetime(2019, 4, 1, 11, 0), datetime.datetime(2019, 4, 1, 11, 15), datetime.datetime(2019, 4, 1, 11, 30), datetime.datetime(2019, 4, 1, 11, 45), datetime.datetime(2019, 4, 1, 12, 0), datetime.datetime(2019, 4, 1, 12, 15), datetime.datetime(2019, 4, 1, 12, 30), datetime.datetime(2019, 4, 1, 12, 45), datetime.datetime(2019, 4, 1, 13, 0), datetime.datetime(2019, 4, 1, 13, 15), date

# change Timezone

In [ ]:
new_timezone = 'America/Los_Angeles'

connectionTime_aware = change_timezone_as_list(connectionTime_utc,new_timezone)

disconnectTime_aware = change_timezone_as_list(disconnectTime_utc ,new_timezone)

doneChargingTime_aware = change_timezone_as_list(doneChargingTime_utc,new_timezone)

zeitreihe_aware = change_timezone_as_list(zeitreihe_utc,new_timezone)

print(zeitreihe_aware)

[datetime.datetime(2019, 4, 1, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 0, 15, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 0, 30, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 0, 45, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 1, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 1, 15, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 1, 30, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 1, 45, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 2, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2019, 4, 1, 2, 15, tzinfo=<DstTzInfo 'America/Lo



change matplotlib timezone

In [ ]:

rcParams['timezone'] = new_timezone

# build Dataframe

In [ ]:
df = pd.DataFrame(list(zip(df_pre_parse['userID'],connectionTime_aware,disconnectTime_aware,doneChargingTime_aware,df_pre_parse['kWhDelivered'])),
                      columns = ['userID','connectionTime','disconnectTime','doneChargingTime','kWhDelivered'])

In [ ]:
df


,userID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered
0,000001083,2019-03-31 12:40:43-07:00,2019-03-31 19:26:57-07:00,2019-03-31 19:26:52-07:00,6.219
1,000000489,2019-03-31 14:16:46-07:00,2019-03-31 14:23:08-07:00,2019-03-31 14:23:03-07:00,0.562
2,000000243,2019-03-31 16:07:33-07:00,2019-03-31 16:46:29-07:00,2019-03-31 16:46:21-07:00,3.870
3,000001093,2019-03-31 16:21:59-07:00,2019-03-31 18:41:26-07:00,2019-03-31 16:45:20-07:00,0.789
4,000000365,2019-03-31 16:35:22-07:00,2019-03-31 17:55:10-07:00,2019-03-31 17:55:02-07:00,4.501
...,...,...,...,...,...
1730,000001152,2019-05-31 21:32:23-07:00,2019-06-01 09:27:29-07:00,2019-06-01 00:55:35-07:00,11.865
1731,000000675,2019-06-01 07:55:43-07:00,2019-06-01 09:44:17-07:00,2019-06-01 09:20:10-07:00,8.082
1732,000001133,2019-06-01 09:59:26-07:00,2019-06-01 11:27:58-07:00,2019-06-01 11:03:24-07:00,6.062
1733,000000365,2019-06-01 10:45:34-07:00,2019-06-01 11:28:51-07:00,2019-06-01 11:28:42-07:00,2.502


#User Id and Max_kWh

In [ ]:
df_userID_kWhDelivered = df[["userID", "kWhDelivered"]]
print(df_userID_kWhDelivered)


#Create a dictionary, using the List items as keys. This will automatically remove any duplicates because dictionaries cannot have duplicate keys.
#Then, convert the dictionary back into a list:
user_id_list = list(dict.fromkeys(df_userID_kWhDelivered['userID'])) 



         userID  kWhDelivered
0     000001083         6.219
1     000000489         0.562
2     000000243         3.870
3     000001093         0.789
4     000000365         4.501
...         ...           ...
1730  000001152        11.865
1731  000000675         8.082
1732  000001133         6.062
1733  000000365         2.502
1734  000000891         1.790

[1735 rows x 2 columns]


In [ ]:
print(user_id_list)

['000001083', '000000489', '000000243', '000001093', '000000365', '000001082', '000000781', '000000560', '000000569', '000001470', '000000562', '000000712', '000000559', '000000579', '000000714', '000000743', '000001534', '000001526', '000000324', '000000676', '000000858', '000000620', '000001481', '000000558', '000000609', '000001087', '000001236', '000001243', '000000891', '000001135', '000000777', '000001164', '000000838', '000001202', '000001104', '000000069', '000000945', '000000515', '000001137', '000000567', '000001957', '000001095', '000000234', '000001001', '000000710', '000001912', '000001746', '000001089', '000001991', '000000850', '000000632', '000000832', '000000774', '000001161', '000002069', '000001099', '000000301', '000001524', '000000347', '000001154', '000000171', '000001953', '000000752', '000001039', '000000867', '000001108', '000001263', '000000818', '000001208', '000001274', '000001124', '000000248', '000001384', '000002192', '000001826', '000001142', '000000675'

function to get Max_kWh for userID

In [ ]:
def get_df_Max_kWh(id_list, df, df_userID, capacity_factor):

  Max_kWh_id_list = []
  for i in range(len(id_list)):
    ID_list = df_userID == id_list[i]
    df_ID_list = df[ID_list]
    Max_kWh_holder = capacity_factor * max(df_ID_list['kWhDelivered'].values)
    Max_kWh_id_list.append(Max_kWh_holder)

  df_Max_kWh = pd.DataFrame(list(zip(id_list,Max_kWh_id_list)),
                      columns = ['userID','battery_capacity[kWh]'])
  
  return df_Max_kWh


use function get Max_kWh for userID

In [ ]:
df_Max_kWh = get_df_Max_kWh(user_id_list,df_userID_kWhDelivered, df_userID_kWhDelivered['userID'],1.25)

In [ ]:
print(df_Max_kWh)

        userID  battery_capacity[kWh]
0    000001083                8.03750
1    000000489               67.22375
2    000000243                8.49000
3    000001093                7.52375
4    000000365                5.62625
..         ...                    ...
156  000001007               13.80125
157  000001272               19.21000
158  000002611                7.34750
159  000001752                6.00875
160  000002607               13.59500

[161 rows x 2 columns]


add new colum to df_userID_kWhDelivered

In [ ]:
df_userID_kWhDelivered['battery_capacity[kWh]'] = ''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


function to add battery_capacity values to dataframe

In [ ]:
def add_value_from_df_to_df_with_same_column_values(df_add, df_to_add,add_same_column,to_add_same_column,add_column, to_add_column):
  for i in range(len(df_add)):
    for j in range(len(df_to_add)):
      if df_add.iloc[i, add_same_column] == df_to_add.iloc[j , to_add_same_column]:
        df_add.iloc[i, add_column] = df_to_add.iloc[j, to_add_column]
  return df_add

In [ ]:
df_userID_kWhDelivered = add_value_from_df_to_df_with_same_column_values(df_userID_kWhDelivered,df_Max_kWh, 0, 0, 2, 1)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
print(df_userID_kWhDelivered)

         userID  kWhDelivered battery_capacity[kWh]
0     000001083         6.219                8.0375
1     000000489         0.562               67.2238
2     000000243         3.870                  8.49
3     000001093         0.789               7.52375
4     000000365         4.501               5.62625
...         ...           ...                   ...
1730  000001152        11.865               19.4738
1731  000000675         8.082                15.235
1732  000001133         6.062               9.86875
1733  000000365         2.502               5.62625
1734  000000891         1.790               7.32875

[1735 rows x 3 columns]


get battery capacity in %

In [ ]:
def get_percent_of_two_columns(df, name_new_column ,new_column_to_add, column_to_divide, column_divide,percent_digits ):
  df[name_new_column] = ''
  for i in range(len(df)):
    df.iloc[i, new_column_to_add] = round(df.iloc[i, column_to_divide]/df.iloc[i,column_divide],percent_digits)*100
  return df

In [ ]:
df_userID_kWhDelivered = get_percent_of_two_columns(df_userID_kWhDelivered, 'battery_capacity[%]',3,1,2,4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
print(df_userID_kWhDelivered)

         userID  kWhDelivered battery_capacity[kWh] battery_capacity[%]
0     000001083         6.219                8.0375               77.37
1     000000489         0.562               67.2238                0.84
2     000000243         3.870                  8.49               45.58
3     000001093         0.789               7.52375               10.49
4     000000365         4.501               5.62625                  80
...         ...           ...                   ...                 ...
1730  000001152        11.865               19.4738               60.93
1731  000000675         8.082                15.235               53.05
1732  000001133         6.062               9.86875               61.43
1733  000000365         2.502               5.62625               44.47
1734  000000891         1.790               7.32875               24.42

[1735 rows x 4 columns]


# pandas to display Max columns of df

In [ ]:
pd.set_option('display.max_columns', 5000)

# Define SoC

difference between connection-disconnect & connection-donecharging 

In [ ]:
def add_new_colum_with_timedifference_of_two_columns_in_min(df, name_new_column, first_time_column, second_time_column, new_column):
  df[name_new_column] = ''
  for i in range(len(df)):
    diff_holder = df.iloc[i, second_time_column] - df.iloc[i,first_time_column]
    diff_holder = diff_holder.seconds//60
    df.iloc[i, new_column] = diff_holder
  return df 

In [ ]:
df_all = df

df_all = add_new_colum_with_timedifference_of_two_columns_in_min(df_all,'diff_con_discon[min]',1,2,5)
df_all = add_new_colum_with_timedifference_of_two_columns_in_min(df_all,'diff_con_done[min]',1,3,6)



if diff_con_done > diff_con_discon then SoC_end = 100%

In [ ]:
df_all['SoC_start[%]'] = ''
df_all['SoC_end[%]'] = ''

for i in range(len(df_all)):
  if df_all.iloc[i,5] >  df_all.iloc[i,6]:
    df_all.iloc[i,8] = 100
  
print(df_all)

         userID            connectionTime            disconnectTime  \
0     000001083 2019-03-31 12:40:43-07:00 2019-03-31 19:26:57-07:00   
1     000000489 2019-03-31 14:16:46-07:00 2019-03-31 14:23:08-07:00   
2     000000243 2019-03-31 16:07:33-07:00 2019-03-31 16:46:29-07:00   
3     000001093 2019-03-31 16:21:59-07:00 2019-03-31 18:41:26-07:00   
4     000000365 2019-03-31 16:35:22-07:00 2019-03-31 17:55:10-07:00   
...         ...                       ...                       ...   
1730  000001152 2019-05-31 21:32:23-07:00 2019-06-01 09:27:29-07:00   
1731  000000675 2019-06-01 07:55:43-07:00 2019-06-01 09:44:17-07:00   
1732  000001133 2019-06-01 09:59:26-07:00 2019-06-01 11:27:58-07:00   
1733  000000365 2019-06-01 10:45:34-07:00 2019-06-01 11:28:51-07:00   
1734  000000891 2019-06-01 11:59:32-07:00 2019-06-01 13:54:28-07:00   

              doneChargingTime  kWhDelivered diff_con_discon[min]  \
0    2019-03-31 19:26:52-07:00         6.219                  406   
1    2019

merge df_all and df_userID_kWhDelivered

In [ ]:
df_all_merged = pd.concat([df_all,df_userID_kWhDelivered],axis=1)


drop duplicated columns

In [ ]:
df_all_merged = df_all_merged.iloc[:,~df_all_merged.columns.duplicated()]
df_all_merged

,userID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,diff_con_discon[min],diff_con_done[min],SoC_start[%],SoC_end[%],battery_capacity[kWh],battery_capacity[%]
0,000001083,2019-03-31 12:40:43-07:00,2019-03-31 19:26:57-07:00,2019-03-31 19:26:52-07:00,6.219,406,406,,,8.0375,77.37
1,000000489,2019-03-31 14:16:46-07:00,2019-03-31 14:23:08-07:00,2019-03-31 14:23:03-07:00,0.562,6,6,,,67.2238,0.84
2,000000243,2019-03-31 16:07:33-07:00,2019-03-31 16:46:29-07:00,2019-03-31 16:46:21-07:00,3.870,38,38,,,8.49,45.58
3,000001093,2019-03-31 16:21:59-07:00,2019-03-31 18:41:26-07:00,2019-03-31 16:45:20-07:00,0.789,139,23,,100,7.52375,10.49
4,000000365,2019-03-31 16:35:22-07:00,2019-03-31 17:55:10-07:00,2019-03-31 17:55:02-07:00,4.501,79,79,,,5.62625,80
...,...,...,...,...,...,...,...,...,...,...,...
1730,000001152,2019-05-31 21:32:23-07:00,2019-06-01 09:27:29-07:00,2019-06-01 00:55:35-07:00,11.865,715,203,,100,19.4738,60.93
1731,000000675,2019-06-01 07:55:43-07:00,2019-06-01 09:44:17-07:00,2019-06-01 09:20:10-07:00,8.082,108,84,,100,15.235,53.05
1732,000001133,2019-06-01 09:59:26-07:00,2019-06-01 11:27:58-07:00,2019-06-01 11:03:24-07:00,6.062,88,63,,100,9.86875,61.43
1733,000000365,2019-06-01 10:45:34-07:00,2019-06-01 11:28:51-07:00,2019-06-01 11:28:42-07:00,2.502,43,43,,,5.62625,44.47


If SoC_end is 100 then subtract battery_capacity for SoC_start

In [ ]:
df_all_merged = pd.DataFrame(df_all_merged)
for i in range(len(df_all_merged)):
  if df_all_merged.iloc[i,8] == 100:
    SoC_start_holder = 100 - df_all_merged.iloc[i,10]
    df_all_merged.iloc[i,7] = SoC_start_holder


use random uniform before and after eleven for SoC start

In [ ]:
SoC_beforeeleven_list = np.random.uniform(60,80,1000)
SoC_aftereleven_list = np.random.uniform(40,60,1000)

time_eleven = dt.strptime('11:00:00','%H:%M:%S')
time_eleven = time_eleven.time()

In [ ]:
for i in range(len(df_all_merged)):
  if df_all_merged.iloc[i,8] != 100 and df_all_merged.iloc[0,1].time() < time_eleven:
    df_all_merged.iloc[i,7] = round(random.choice(SoC_beforeeleven_list),2)
    df_all_merged.iloc[i,8] = df_all_merged.iloc[i,7] + df_all_merged.iloc[i,10]
  elif df_all_merged.iloc[i,8] != 100:
    df_all_merged.iloc[i,7] = round(random.choice(SoC_aftereleven_list),2)
    df_all_merged.iloc[i,8] = df_all_merged.iloc[i,7] + df_all_merged.iloc[i,10]
  
  if df_all_merged.iloc[i,8] > 100:
    df_all_merged.iloc[i,7] = 100 - df_all_merged.iloc[i,10]
    df_all_merged.iloc[i,8] = 100

In [ ]:
df_all_merged

,userID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,diff_con_discon[min],diff_con_done[min],SoC_start[%],SoC_end[%],battery_capacity[kWh],battery_capacity[%]
0,000001083,2019-03-31 12:40:43-07:00,2019-03-31 19:26:57-07:00,2019-03-31 19:26:52-07:00,6.219,406,406,22.63,100,8.0375,77.37
1,000000489,2019-03-31 14:16:46-07:00,2019-03-31 14:23:08-07:00,2019-03-31 14:23:03-07:00,0.562,6,6,48.05,48.89,67.2238,0.84
2,000000243,2019-03-31 16:07:33-07:00,2019-03-31 16:46:29-07:00,2019-03-31 16:46:21-07:00,3.870,38,38,41.31,86.89,8.49,45.58
3,000001093,2019-03-31 16:21:59-07:00,2019-03-31 18:41:26-07:00,2019-03-31 16:45:20-07:00,0.789,139,23,89.51,100,7.52375,10.49
4,000000365,2019-03-31 16:35:22-07:00,2019-03-31 17:55:10-07:00,2019-03-31 17:55:02-07:00,4.501,79,79,20,100,5.62625,80
...,...,...,...,...,...,...,...,...,...,...,...
1730,000001152,2019-05-31 21:32:23-07:00,2019-06-01 09:27:29-07:00,2019-06-01 00:55:35-07:00,11.865,715,203,39.07,100,19.4738,60.93
1731,000000675,2019-06-01 07:55:43-07:00,2019-06-01 09:44:17-07:00,2019-06-01 09:20:10-07:00,8.082,108,84,46.95,100,15.235,53.05
1732,000001133,2019-06-01 09:59:26-07:00,2019-06-01 11:27:58-07:00,2019-06-01 11:03:24-07:00,6.062,88,63,38.57,100,9.86875,61.43
1733,000000365,2019-06-01 10:45:34-07:00,2019-06-01 11:28:51-07:00,2019-06-01 11:28:42-07:00,2.502,43,43,54.78,99.25,5.62625,44.47


# Save df_all_merged to *CSV*

In [ ]:
#df_all_merged.to_csv(r'/content/drive/My Drive/Studienarbeit/df_all_merged.csv')

# Find SoC and kWh for the location with potential kWh

In [ ]:
connectionTime = df_all_merged['connectionTime']

disconnectTime = df_all_merged['disconnectTime']

function for current kWh for the location

In [ ]:
def current_kWh_for_location(df, timeline, start_time, end_time,SoC_start_column, battery_capacity_column, kWhDelivered_column ):
  car_parked = []
  parked_kWh_SoC = []

  j= -1
  for i in range(len(df)): #df
    j = j + 1
    car_parked = []
    parked_kWh_SoC = []
    parked_kWh_Max = []

    for i in range(len(timeline)): #timeline
      if start_time[j] <= timeline[i] and end_time[j] > timeline[i]: #start_time & end_time
        car_parked.append(1)
      else:
        car_parked.append(0)

    if sum(car_parked) > 0:
      SoC_start_holder = round(SoC_start_column[j]*battery_capacity_column[j]/100 ,2) #Soc_start_column * baterry_capacity_column/100
      kWh_holder = round(kWhDelivered_column[j]/sum(car_parked),2)# kWhDelivered_column & with sum(9car_parked) linear interpolation
      parked_kWh_SoC = [i* SoC_start_holder for i in car_parked] # multiplicate two lists
      parked_kWh_Max = [i* battery_capacity_column[j] for i in car_parked] # potential kWh capacity
  
      for i in range(len(parked_kWh_SoC)):
        if parked_kWh_SoC[i-1] != 0 and parked_kWh_SoC[i] != 0:
          parked_kWh_SoC[i] = parked_kWh_SoC[i-1] + kWh_holder

    else:
      parked_kWh_SoC = [0] * len(car_parked) #list add only with same len
      parked_kWh_Max = [0] *len(car_parked)

    if j <= 0: # how in for loop ?
      parked_cars_len = [0] * len(car_parked) #list add only with same len
      parked_kWh_SoC = [0] * len(car_parked)
      parked_kWh_Max = [0] *len(car_parked)

      sum_parked_count = [a + b for a, b in zip(car_parked, parked_cars_len)] # add list together
      sum_kWh_count = [a + b for a, b in zip(parked_kWh_SoC, parked_cars_len)]
      sum_kWh_potential = [a + b for a, b in zip(parked_kWh_Max, parked_cars_len)]
  
    else:
      sum_parked_count = [a + b for a, b in zip(car_parked, sum_parked_count)]
      sum_kWh_count = [a + b for a, b in zip(parked_kWh_SoC, sum_kWh_count)]
      sum_kWh_potential = [a + b for a, b in zip(parked_kWh_Max, sum_kWh_potential)]
  
  sum_kWh_count = [round(num, 2) for num in sum_kWh_count] # round to 2 digits
  sum_kWh_potential = [round(num, 2) for num in sum_kWh_potential] # round to 2 digits

  df_current_kWh = pd.DataFrame(list(zip(timeline,sum_parked_count,sum_kWh_count, sum_kWh_potential)),
                      columns = ['zeitreihe','sum_parked_count', 'sum_kWh_count','sum_kWh_potential'])
  return df_current_kWh

In [ ]:
df_sum = current_kWh_for_location(df_all_merged, zeitreihe_aware, connectionTime, disconnectTime, df_all_merged['SoC_start[%]'],df_all_merged['battery_capacity[kWh]'],df_all_merged['kWhDelivered'])

# df_sum for Plot and mean

add time,dayofweek and date to new column in df

In [ ]:
df_sum['timeOfDay'] = df_sum['zeitreihe'].dt.time

df_sum['dayOfWeek'] = df_sum['zeitreihe'].dt.dayofweek #df.day_name() Monday,...

df_sum['date'] = df_sum['zeitreihe'].dt.date


get the dayofweek from code to initial

In [ ]:
days = {0:'Mo', 1:'Tu', 2:'We', 3:'Th', 4:'Fr', 5:'Sa', 6:'Su'} 
df_sum['dayOfWeek'] = df_sum['dayOfWeek'].apply(lambda x: days[x]) #In order to print the days in a particular format.


In [ ]:
df_sum

,zeitreihe,sum_parked_count,sum_kWh_count,sum_kWh_potential,timeOfDay,dayOfWeek,date
0,2019-04-01 00:00:00-07:00,1,23.52,67.22,00:00:00,Mo,2019-04-01
1,2019-04-01 00:15:00-07:00,1,24.77,67.22,00:15:00,Mo,2019-04-01
2,2019-04-01 00:30:00-07:00,1,26.02,67.22,00:30:00,Mo,2019-04-01
3,2019-04-01 00:45:00-07:00,1,27.27,67.22,00:45:00,Mo,2019-04-01
4,2019-04-01 01:00:00-07:00,1,28.52,67.22,01:00:00,Mo,2019-04-01
...,...,...,...,...,...,...,...
5852,2019-05-31 23:00:00-07:00,4,77.20,87.84,23:00:00,Fr,2019-05-31
5853,2019-05-31 23:15:00-07:00,4,79.28,87.84,23:15:00,Fr,2019-05-31
5854,2019-05-31 23:30:00-07:00,4,81.36,87.84,23:30:00,Fr,2019-05-31
5855,2019-05-31 23:45:00-07:00,4,83.44,87.84,23:45:00,Fr,2019-05-31
